In [ ]:
# === Monta Google Drive ===
from google.colab import drive
drive.mount('/content/drive')
 
# === Importa funzione get_all_tickers da my_tickers.py ===
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks1')
from my_tickers import get_all_tickers
 
import pandas as pd
import yfinance as yf
import numpy as np
import warnings
from datetime import datetime
import os
 
warnings.simplefilter('ignore', category=FutureWarning)
 
# === Parametri principali ===
poc_period = '5y'
soglia_poc = 5
filter_start_date = pd.to_datetime("2000-01-01")
 
# === Funzioni storiche ===
def get_hist(ticker, period):
    try:
        df = yf.download(ticker, period=period, progress=False)
        return df
    except Exception as e:
        print(f"Errore storico {ticker}: {e}")
        return pd.DataFrame()
 
def calculate_drawdowns(prices):
    if prices.empty:
        return np.nan, np.nan, np.nan
    cummax = prices.cummax()
    drawdown = (cummax - prices) / cummax * 100
    return drawdown.max(), drawdown.mean(), drawdown.iloc[-1]
 
def get_poc_daily(ticker, period="5y", bins=200):
    try:
        df = yf.download(ticker, period=period, interval="1d", progress=False, auto_adjust=False)
        if df.empty:
            return None
    except Exception as e:
        print(f"Errore download POC data for {ticker}: {e}")
        return None
 
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ["_".join(map(str, col)).strip() for col in df.columns]
 
    high_col = next((c for c in df.columns if 'high' in c.lower()), None)
    low_col = next((c for c in df.columns if 'low' in c.lower()), None)
    vol_col = next((c for c in df.columns if 'volume' in c.lower()), None)
 
    if high_col is None or low_col is None or vol_col is None:
        return None
 
    price_min = df[low_col].min()
    price_max = df[high_col].max()
    if price_min == price_max:
        return None
 
    price_bins = np.linspace(price_min, price_max, bins)
    volume_profile = np.zeros(len(price_bins) - 1)
 
    for _, row in df.iterrows():
        if row[high_col] > row[low_col] and row[vol_col] > 0:
            mask = (price_bins[:-1] >= row[low_col]) & (price_bins[:-1] <= row[high_col])
            idx = np.where(mask)[0]
            if len(idx) > 0:
                vol_share = row[vol_col] / len(idx)
                volume_profile[idx] += vol_share
 
    if volume_profile.sum() == 0:
        return None
 
    poc_index = np.argmax(volume_profile)
    return (price_bins[poc_index] + price_bins[poc_index + 1]) / 2
 
# === Recupera tutti i ticker con indice ===
ticker_dict = get_all_tickers(flat=False)
ticker_to_index = {}
for idx_name, tickers in ticker_dict.items():
    for t in tickers:
        if t in ticker_to_index:
            ticker_to_index[t] += f", {idx_name}"
        else:
            ticker_to_index[t] = idx_name
 
all_tickers = list(ticker_to_index.keys())
print(f"Trovati {len(all_tickers)} ticker tra tutti gli indici")
 
# === Ciclo principale sui ticker ===
risultati = []
 
for ticker in all_tickers:
    try:
        poc_price = get_poc_daily(ticker, period=poc_period)
        if poc_price is None:
            continue
 
        df_hist = get_hist(ticker, period="1d")
        if df_hist.empty or "Close" not in df_hist.columns:
            continue
        current_price = float(df_hist["Close"].iloc[-1])
 
        distanza_poc = (current_price - poc_price) / poc_price * 100
 
        if abs(distanza_poc) <= soglia_poc:
            df_all = get_hist(ticker, period="max")
            if df_all.empty or "Close" not in df_all.columns:
                continue
            df_filtered = df_all[df_all.index >= filter_start_date].copy()
            if df_filtered.empty:
                continue
 
            close_prices = df_filtered["Close"]
            all_time_high = close_prices.max()
            max_dd, avg_dd, current_dd = calculate_drawdowns(close_prices)
 
            risultati.append({
                "Ticker": ticker,
                "Indice": ticker_to_index[ticker],
                "POC": poc_price,
                "Prezzo Attuale": current_price,
                "Distanza POC %": distanza_poc,
                "All Time High": float(all_time_high),
                "Max Drawdown %": float(max_dd),
                "Avg Drawdown %": float(avg_dd),
                "Current Drawdown %": float(current_dd)
            })
 
    except Exception as e:
        print(f"Errore con {ticker}: {e}")
        continue
 
# === Risultati ===
df_risultati = pd.DataFrame(risultati)
 
if df_risultati.empty:
    print("⚠ Nessun titolo ha passato i filtri sulla distanza dal POC o non ha dati storici sufficienti.")
else:
    df_risultati = df_risultati.sort_values(by="Current Drawdown %", ascending=False)
   
    # Stampare l'intero DataFrame come testo
    print(df_risultati.to_string())
 
    # === Salvataggio file Excel con numero settimana corrente ===
    week_number = datetime.now().isocalendar()[1]
    poc_folder = '/content/drive/MyDrive/POC'
    os.makedirs(poc_folder, exist_ok=True)
    file_path = os.path.join(poc_folder, f'POC_week_{week_number}.xlsx')
    df_risultati.to_excel(file_path, index=False)
    print(f"\n✅ File salvato in: {file_path}")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === Esecuzione sul ticker selezionato ===
# Sostituisci 'AAPL' con il ticker che ti interessa
ticker_da_analizzare = 'PYPL'

# === Parametri (mantieni se vuoi personalizzare da qui) ===
poc_period = '5y'  # Periodo storico per calcolo POC
soglia_poc = 5     # distanza massima % dal POC
filter_start_date = pd.Timestamp("2000-01-01") # Data di inizio per il filtro storico in analyze_stock


# =====================
# FUNZIONI DI SUPPORTO (da cella TQLvO0vxSg-n)
# =====================

def get_hist(ticker, period="max", adjusted=False):
    """
    Scarica dati storici da Yahoo Finance.
    Appiattisce le colonne MultiIndex se presenti.
    """
    try:
        df = yf.download(
            ticker,
            period=period,
            interval="1d",
            progress=False,
            auto_adjust=adjusted
        )

        # Appiattisci MultiIndex colonne
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [col[0] for col in df.columns]

        # Controllo colonne fondamentali
        required_cols = {"Open", "High", "Low", "Close", "Volume"}
        if not required_cols.issubset(df.columns):
            print(f"⚠ Dati incompleti per {ticker}, colonne trovate: {df.columns.tolist()}")
            return pd.DataFrame()

        return df
    except Exception as e:
        print(f"Errore durante il download dei dati per {ticker}: {e}")
        return pd.DataFrame()

def calculate_poc(df, bins=200):
    """
    Calcola il Point of Control (POC) distribuendo il volume tra High-Low di ciascuna candela.
    Il volume di ogni giorno viene distribuito uniformemente sui "bins" di prezzo
    compresi tra il prezzo minimo (Low) e il prezzo massimo (High) di quella giornata.
    """
    if df.empty:
        print("POC non calcolabile: dataframe vuoto")
        return None

    price_min = df["Low"].min()
    price_max = df["High"].max()
    if price_min == price_max: # Aggiunto controllo per prezzi costanti
        print(f"Prezzi costanti per calcolo POC: {price_min}")
        return None


    price_bins = np.linspace(price_min, price_max, bins)
    volume_profile = np.zeros(len(price_bins) - 1)

    for _, row in df.iterrows():
        if row["High"] > row["Low"] and row["Volume"] > 0:
            # Trova gli indici dei bin coperti dal range High-Low
            # Usiamo searchsorted per maggiore efficienza con grandi dataframes
            low_bin_idx = np.searchsorted(price_bins, row["Low"], side='right') - 1
            high_bin_idx = np.searchsorted(price_bins, row["High"], side='left')

            # Assicurati che gli indici siano validi e che low <= high
            low_bin_idx = max(0, min(low_bin_idx, len(price_bins) - 2))
            high_bin_idx = max(0, min(high_bin_idx, len(price_bins) - 1))


            if high_bin_idx > low_bin_idx:
                bins_covered = np.arange(low_bin_idx, high_bin_idx)
                if len(bins_covered) > 0:
                    vol_share = row["Volume"] / len(bins_covered)
                    volume_profile[bins_covered] += vol_share
            elif high_bin_idx == low_bin_idx and low_bin_idx < len(price_bins) -1: # Handle case where High and Low fall in the same bin
                 volume_profile[low_bin_idx] += row["Volume"] # Add all volume to that bin


    if volume_profile.sum() == 0:
        print("POC non calcolabile: volume totale zero")
        return None

    poc_index = np.argmax(volume_profile)
    # Assicurati che poc_index sia un indice valido per price_bins
    if poc_index >= len(price_bins) -1:
        poc_index = len(price_bins) - 2 # Fallback to the last valid bin midpoint


    poc_price = (price_bins[poc_index] + price_bins[poc_index + 1]) / 2
    return poc_price, price_bins[:-1], volume_profile


def calculate_drawdown(prices):
    """Calcola drawdown massimo e periodo relativo"""
    if prices.empty:
        return None, None, None, None

    rolling_max = prices.cummax()
    drawdown = (prices - rolling_max) / rolling_max
    min_dd = drawdown.min()
    end_date = drawdown.idxmin()
    # Trova l'indice del massimo prima o all'end_date
    # Usiamo idxmax() con un filtro per evitare massimi successivi
    start_date = prices.loc[:end_date].idxmax()
    return drawdown, min_dd, start_date, end_date

def calculate_ath_distance(prices):
    """Calcola l'All Time High e distanza % dal prezzo corrente"""
    if prices.empty:
        return None, None
    ath = prices.max()
    current_price = prices.iloc[-1]
    distance = (current_price - ath) / ath
    return ath, distance

# =====================
# FUNZIONE PRINCIPALE (da cella TQLvO0vxSg-n)
# =====================

def analyze_stock(ticker, poc_period="5y", filter_start_date=pd.Timestamp("2010-01-01")):
    print(f"\nAnalisi per {ticker}")

    # --- Dati RAW (non adjusted) per POC ---
    df_raw = get_hist(ticker, period=poc_period, adjusted=False)
    if df_raw.empty:
        print(f"⚠ Dati insufficienti per POC su {ticker}")
        return

    # --- Dati ADJUSTED per drawdown, ATH, prezzo ---
    df_adj = get_hist(ticker, period="max", adjusted=True)
    if df_adj.empty:
        print(f"⚠ Dati insufficienti per analisi prezzo su {ticker}")
        return

    # --- Filtra dati adjusted per la data di inizio specificata ---
    df_adj_filtered = df_adj[df_adj.index >= filter_start_date].copy()

    if df_adj_filtered.empty:
        print(f"⚠ Nessun dato disponibile per {ticker} dopo il {filter_start_date.date()}")
        return


    # --- Calcolo POC e Volume Profile ---
    poc_result = calculate_poc(df_raw, bins=200)
    poc_price = None
    price_bins = None
    volume_profile = None

    if poc_result is not None:
        poc_price, price_bins, volume_profile = poc_result
    else:
        print(f"POC e Volume Profile non calcolabili per {ticker} nel periodo {poc_period}")


    # --- Prezzo corrente e performance (basata sui dati filtrati) ---
    current_price = df_adj_filtered["Close"].iloc[-1]
    start_price = df_adj_filtered["Close"].iloc[0]
    perf = (current_price - start_price) / start_price

    # --- Drawdown (basato sui dati filtrati) ---
    drawdown, max_dd, start_dd, end_dd = calculate_drawdown(df_adj_filtered["Close"])

    # --- ATH (basato sui dati filtrati) ---
    ath, dist_ath = calculate_ath_distance(df_adj_filtered["Close"])

    # --- Distanza da POC ---
    dist_poc = None # Inizializza a None
    if poc_price is not None: # Calcola la distanza solo se il POC è stato calcolato
       dist_poc = (current_price - poc_price) / poc_price


    # --- Stampa risultati ---
    print(f"Prezzo corrente: {current_price:.2f}")
    print(f"Performance dal {filter_start_date.date()}: {perf:.2%}")
    # Stampa risultati drawdown solo se disponibili
    if max_dd is not None:
        print(f"Max Drawdown ({filter_start_date.date()} onwards): {max_dd:.2%} (da {start_dd.date()} a {end_dd.date()})")
    else:
        print(f"Max Drawdown ({filter_start_date.date()} onwards): Non calcolabile")

    # Stampa risultati ATH solo se disponibili
    if ath is not None and dist_ath is not None:
        print(f"ATH ({filter_start_date.date()} onwards): {ath:.2f} | Distanza attuale: {dist_ath:.2%}")
    else:
         print(f"ATH ({filter_start_date.date()} onwards): Non calcolabile")

    # Stampa risultati POC solo se disponibili
    if poc_price is not None and dist_poc is not None:
        print(f"POC ({poc_period} period): {poc_price:.2f} | Distanza da POC: {dist_poc:.2%}")
    else:
        print(f"POC ({poc_period} period): Non calcolabile")


    # --- Grafico con Volume Profile ---
    fig, ax1 = plt.subplots(figsize=(12,6))

    # Plot del prezzo
    ax1.plot(df_adj_filtered.index, df_adj_filtered["Close"], label="Prezzo (Adj Close)", color="blue")
    ax1.set_ylabel("Prezzo")

    # Linee POC e ATH
    if poc_price is not None:
        ax1.axhline(poc_price, color="red", linestyle="--", label=f"POC {poc_price:.2f} ({poc_period})")
    if ath is not None:
        ax1.axhline(ath, color="green", linestyle="--", label=f"ATH {ath:.2f} ({filter_start_date.date()} onwards)")

    # Crea un secondo asse Y per il Volume Profile
    ax2 = ax1.twiny()

    # Plot del Volume Profile (come istogramma orizzontale)
    if price_bins is not None and volume_profile is not None:
        # Normalizza il volume profile per adattarlo al grafico del prezzo
        # Scaliamo il volume profile in modo che si estenda per una frazione della larghezza del grafico
        volume_scale = 0.2 # Puoi aggiustare questo valore
        normalized_volume = volume_profile / volume_profile.max() * (df_adj_filtered.index[-1] - df_adj_filtered.index[0]) * volume_scale

        # Per l'istogramma orizzontale, usiamo `barh`
        # L'asse x sarà il volume normalizzato, l'asse y saranno i centri dei bin di prezzo
        bin_centers = (price_bins + np.diff(price_bins)[0] / 2)
        # Filtra i bin_centers per la lunghezza del volume_profile
        bin_centers = bin_centers[:len(volume_profile)]


        ax2.barh(bin_centers, normalized_volume, height=np.diff(price_bins)[0], color='gray', alpha=0.5, label="Volume Profile")
        ax2.set_xlabel("Volume Profile (scala arbitraria)")


    ax1.set_title(f"{ticker} - Prezzo, POC, ATH e Volume Profile")
    ax1.legend(loc='upper left') # Sposta la legenda per evitare sovrapposizioni
    if volume_profile is not None:
        ax2.legend(loc='upper right') # Legenda per il volume profile

    # Imposta i limiti dell'asse x per il prezzo per far spazio al volume profile
    # Potresti voler aggiustare questo in base a quanto spazio vuoi dedicare al volume profile
    # ax1.set_xlim(df_adj_filtered.index[0] - (df_adj_filtered.index[-1] - df_adj_filtered.index[0]) * volume_scale, df_adj_filtered.index[-1])
    # ax2.set_xlim(0, normalized_volume.max()) # Imposta i limiti per l'asse del volume profile

    plt.show()


# =====================
# ESECUZIONE
# =====================
analyze_stock(ticker_da_analizzare, poc_period=poc_period, filter_start_date=filter_start_date)